In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import re
import os
import glob
import pandas as pd
import numpy as np
import seaborn as sns
from numpy import sin, pi

In [ ]:
from mpl_toolkits.axes_grid.anchored_artists import AnchoredText

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline

In [ ]:
sns.set_style('ticks')
sns.set_context('notebook', font_scale=1.2)

In [ ]:
def parse_fort16(f):
    data = []
    idx = []
    for line in f:
        line = line.strip()
        if line[0] not in ('#', '@') and line != 'END':
            parts = [float(x) for x in line.split()]
            idx.append(parts[0])
            data.append(parts[1])
    return idx, data

In [ ]:
def rutherford(en, angle, Z1, Z2):
    angle = angle * pi / 180.
    return (Z1 * Z2 * 1.44 / (4 * en))**2 / sin(angle / 2)**4 * 10

In [ ]:
rootdir = os.path.abspath('/Volumes/projects/phy982/Group_JB_CI/HW3')
# graphdir = os.path.abspath('/Users/josh/Documents/Code/reactions/hw2/images')

In [ ]:
f201files = []
f202files = []
for root, dirnames, filenames in os.walk(rootdir):
    if 'fort.201' in filenames:
        f201files.append(os.path.join(root, 'fort.201'))
    if 'fort.202' in filenames:
        f202files.append(os.path.join(root, 'fort.202'))

In [ ]:
f202files

In [ ]:
dfs = []
for f in f202files:
    name = '_'.join(re.match(rootdir + '/(.*)', os.path.dirname(f)).group(1).split('/'))
    with open(f) as fp:
        idx, data = parse_fort16(fp)
    dfs.append(pd.DataFrame(data, columns=(name,), index=idx))
        
xsecs = pd.concat(dfs, axis=1)

# Data

In [ ]:
expdata = pd.read_table(rootdir + '/data56.dat', delim_whitespace=True, index_col='angle')
expdata['xsec']  *= 1e-3 #/ rutherford(56, expdata.index.values, 1, 20)
expdata['error'] *= 1e-3 #/ rutherford(56, expdata.index.values, 1, 20)

In [ ]:
expdata11 = pd.read_table(rootdir + '/data11.dat', delim_whitespace=True, index_col='angle')

In [ ]:
She = 0.85
xsecs['scaled'] = xsecs.finite_fullrem * She

In [ ]:
xsecs.zero_range.plot(label='Zero-Range', style='')
xsecs.finite_norem.plot(label='Finite-Range (Post)')
xsecs.finite_fullrem.plot(label='Finite-Range w/ Full Complex Remnant (Post)')
xsecs.prior_fullrem.plot(label='Finite-Range w/ Full Complex Remnant (Prior)')
plt.errorbar(expdata.index, expdata.xsec, fmt='k.', yerr=expdata.error, label='Uozumi et al. (1994)')
# plt.semilogy()
plt.grid(False)
plt.legend(frameon=False)
plt.xlabel('CM Angle [Degrees]')
plt.ylabel(r'$d\sigma/d\Omega$ [mb/sr]')
plt.xlim(0, 100)

at = AnchoredText(r"E = 56 MeV",
                  prop=dict(size=18), frameon=False,
                  loc=3,
                  )
plt.gca().add_artist(at)

plt.savefig('highen.pdf')

In [ ]:
xsecs.finite_fullrem.plot(label='Finite-Range w/ Full Complex Remnant (Post)')
xsecs.scaled.plot(label='Scaled with spectroscopic factor')
plt.errorbar(expdata.index, expdata.xsec, fmt='k.', yerr=expdata.error, label='Uozumi et al. (1994)')
# plt.semilogy()
plt.grid(False)
plt.legend(frameon=False)
plt.xlabel('CM Angle [Degrees]')
plt.ylabel(r'$d\sigma/d\Omega$ [mb/sr]')
plt.xlim(0, 100)

at = AnchoredText(r"E = 56 MeV",
                  prop=dict(size=18), frameon=False,
                  loc=3,
                  )

at2 = AnchoredText("S = {}".format(She),
                  prop=dict(size=18), frameon=False,
                  loc=5,
                  )
plt.gca().add_artist(at)
plt.gca().add_artist(at2)

plt.savefig('highen_sfactor.pdf')

In [ ]:
Sle = 0.70
xsecs['lowen_scaled'] = Sle * xsecs.lowen_finite_fullrem

In [ ]:
xsecs.lowen_zero_range.plot(label='Zero-Range')
xsecs.lowen_finite_norem.plot(label='Finite-Range (Post)')
xsecs.lowen_finite_fullrem.plot(label='Finite-Range w/ Full Complex Remnant (Post)')
xsecs.lowen_prior_fullrem.plot(label='Finite-Range w/ Full Complex Remnant (Prior)')
plt.errorbar(expdata11.index, expdata11.xsec, fmt='k.', yerr=expdata11.error, label='Schmidt-Rohr et al. (1964)')
# plt.semilogy()
plt.grid(False)
plt.legend(frameon=False, loc=1)
plt.xlabel('CM Angle [Degrees]')
plt.ylabel(r'$d\sigma/d\Omega$ [mb/sr]')

at = AnchoredText(r"E = 11.8 MeV",
                  prop=dict(size=18), frameon=False,
                  loc=3,
                  )
plt.gca().add_artist(at)

plt.savefig('lowen.pdf')

In [ ]:
xsecs.lowen_finite_fullrem.plot(label='Finite-Range w/ Full Complex Remnant (Post)')
xsecs.lowen_scaled.plot(label='Scaled with spectroscopic factor')
plt.errorbar(expdata11.index, expdata11.xsec, fmt='k.', yerr=expdata11.error, label='Schmidt-Rohr et al. (1964)')
# plt.semilogy()
plt.grid(False)
plt.legend(frameon=False, loc=1)
plt.xlabel('CM Angle [Degrees]')
plt.ylabel(r'$d\sigma/d\Omega$ [mb/sr]')

at = AnchoredText(r"E = 11.8 MeV",
                  prop=dict(size=18), frameon=False,
                  loc=3,
                  )
at2 = AnchoredText("S = {:.2f}".format(Sle),
                  prop=dict(size=18), frameon=False,
                  loc=5,
                  )
plt.gca().add_artist(at)
plt.gca().add_artist(at2)

plt.savefig('lowen_sfactor.pdf')

In [ ]:
exp_spl = InterpolatedUnivariateSpline(expdata.index, expdata.xsec)

In [ ]:
xs = np.linspace(0, 50, 100)

In [ ]:
plt.plot(xs, exp_spl(xs))
plt.errorbar(expdata.index, expdata.xsec, fmt='k.', yerr=expdata.error, label='Uozumi et al. (1994)')

In [ ]:
inrange = xsecs[0:50]

In [ ]:
plt.plot(exp_spl(inrange.index) / inrange.finite_fullrem)